In [118]:
import os
import pandas as pd
import numpy as np
import librosa
from os import listdir
from os.path import isfile, join, normpath
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [119]:
RANDOM_STATE = 26
TEST_SIZE = 0.3
N_MFCC = 2
MIX_CSV_PATH = "../data/HLS-CMDS/Mix.csv"
MIX_AUDIO_PATH = "../data/HLS-CMDS/Mix/Mix"

In [120]:
df = pd.read_csv(MIX_CSV_PATH)
print(df.head())

  Gender      Heart Sound Type  Lung Sound Type Location Heart Sound ID  \
0      F  Late Systolic Murmur          Rhonchi     LUSB          H0001   
1      F                    S3           Normal      RLA          H0002   
2      M   Atrial Fibrillation           Normal      LMA          H0003   
3      F                    S3  Coarse Crackles     Apex          H0004   
4      M              AV Block    Fine Crackles     RUSB          H0005   

  Lung Sound ID Mixed Sound ID  
0         L0001          M0001  
1         L0002          M0002  
2         L0003          M0003  
3         L0004          M0004  
4         L0005          M0005  


In [127]:
df.columns

Index(['Gender', 'Heart Sound Type', 'Lung Sound Type', 'Location',
       'Heart Sound ID', 'Lung Sound ID', 'Mixed Sound ID', 'audio_path'],
      dtype='object')

In [121]:
df['audio_path'] = df['Mixed Sound ID'].apply(lambda x: normpath(join(MIX_AUDIO_PATH, f"{str(x).strip()}.wav").replace('\\', '/')))

In [122]:
print(df.head())

  Gender      Heart Sound Type  Lung Sound Type Location Heart Sound ID  \
0      F  Late Systolic Murmur          Rhonchi     LUSB          H0001   
1      F                    S3           Normal      RLA          H0002   
2      M   Atrial Fibrillation           Normal      LMA          H0003   
3      F                    S3  Coarse Crackles     Apex          H0004   
4      M              AV Block    Fine Crackles     RUSB          H0005   

  Lung Sound ID Mixed Sound ID                          audio_path  
0         L0001          M0001  ..\data\HLS-CMDS\Mix\Mix\M0001.wav  
1         L0002          M0002  ..\data\HLS-CMDS\Mix\Mix\M0002.wav  
2         L0003          M0003  ..\data\HLS-CMDS\Mix\Mix\M0003.wav  
3         L0004          M0004  ..\data\HLS-CMDS\Mix\Mix\M0004.wav  
4         L0005          M0005  ..\data\HLS-CMDS\Mix\Mix\M0005.wav  


In [123]:
def extract_mfcc(filename, n_mfcc=N_MFCC):
    y, sr = librosa.load(filename, sr=166)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    return np.concatenate([mfcc_mean, mfcc_std])

def flatten_mfcc(mfcc):
    return np.array(mfcc).flatten()

In [124]:
mfcc_list = []
labels = []

for _, row in df.iterrows():
    try:
        mfcc = extract_mfcc(row['audio_path'])
        # mfcc_flat = flatten_mfcc(mfcc)
        mfcc_list.append(mfcc)
        labels.append(row['Location'])
    except Exception as e:
        print(f"[WARN] Failed: {row['audio_path']} -> {e}")
    
X = pd.DataFrame(mfcc_list)
y = pd.Series(labels)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

In [125]:
pipe_model = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, class_weight='balanced'))
])

param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf', 'linear', 'poly']
}

model_grid = GridSearchCV(
    estimator=pipe_model, 
    param_grid=param_grid, 
    scoring='f1_macro', 
    cv=5, 
    n_jobs=-1, 
    verbose=2
)

In [126]:
model_grid.fit(Xtrain, ytrain)
model = model_grid.best_estimator_
print(f"Best parameters: {model_grid.best_params_}")

ypred = model.predict(Xtest)
f1 = f1_score(ytest, ypred, average='macro')
print(f"Mix: Location -> macro-F1: {f1}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'svc__C': 100, 'svc__gamma': 0.001, 'svc__kernel': 'linear'}
Mix: Location -> macro-F1: 0.08584239834239833
